# SQL tasks

1. What is the average physical illness in the data?
2. Among smoker vs. non-smoker, please calculate the following metrics:
    1. the amount of diabetes vs. non diabetes can be found in the data?
    2. the avg, min and max physical illness can be found in the data?
3. How many (absolute) smokers do we have that are having serious difficulty walking or climbing stairs as well?
4. What is the distribution (in %) in diabetes among education categories and men, out of all men smokers?
5. Some hospitals had approached you and are asking for some information. Please help them get the following -
    1. Please generate a table of each education level, and the top 5 persons (by ID), where the persons are ranked according to their BMI, DESC
    2. Let’s create a new metric per each person, called “BMIToMentalActivityRatio” - calculated as: BMI / mental activity.
    Note: If one of the “base metrics” is 0, put a 0 to that specific record in our new metric column.
    Once we have this column, let’s understand per each income level, who are the top 7 persons with the highest ratio

### 1. What is the average physical illness in the data?

In [1]:
-- Start writing your SQL query here 

SELECT avg(PhysHlth) as avg
FROM enriched_data
WHERE PhysHlth > 0;

,avg
0,11.493688


### 2. Among smoker vs. non-smoker, please calculate the following metrics:
    1. the amount of diabetes vs. non diabetes can be found in the data?
    2. the avg, min and max physical illness can be found in the data?

In [7]:
-- the amount of diabetes vs. non diabetes can be found in the data?
SELECT SUM(CASE WHEN Diabetes_binary = 1 THEN 1 ELSE 0 END) AS Diabetic,
SUM(CASE WHEN Diabetes_binary = 0 THEN 1 ELSE 0 END) AS Non_Diabetic
FROM id_label;

,diabetic,non_diabetic
0,35346,218334


In [2]:
-- the avg, min and max physical illness can be found in the data?
SELECT AVG(PhysHlth) AS avg, MIN(PhysHlth) AS min, MAX(PhysHlth) AS max
FROM enriched_data;

,avg,min,max
0,4.242081,0,30


### 3. How many (absolute) smokers do we have that are having serious difficulty walking or climbing stairs as well?

In [8]:
SELECT COUNT(DISTINCT e.uniqueID)
FROM enriched_data e 
JOIN health_measures h
ON e.uniqueID = h.uniqueID
WHERE e.Smoker = 1 AND h.DiffWalk = 1;

,count
0,24685


### 4. What is the distribution (in %) in diabetes among education categories and men, out of all men smokers?

In [23]:
WITH t1 as (
SELECT education as education, COUNT(diabetes_binary) + 0.0 as ttl_diabetic
FROM demographic_data d
JOIN id_label i
ON d.uniqueid = i.uniqueid
JOIN health_measures h
ON i.uniqueid = h.uniqueid
JOIN enriched_data e
ON h.uniqueid = e.uniqueid
WHERE diabetes_binary = 1
GROUP BY 1
),
t2 as (
SELECT COUNT(
CASE WHEN diabetes_binary = 0
THEN 1 ELSE 0 END) as ttl_non_diabetic
FROM id_label
),   
t3 as (
SELECT SUM(smoker) + 0.0 as ttl_smoker
FROM demographic_data d
JOIN id_label i
ON d.uniqueid = i.uniqueid
JOIN health_measures h
ON i.uniqueid = h.uniqueid
JOIN enriched_data e
ON h.uniqueid = e.uniqueid
WHERE e.smoker = 1
AND d.sex = 1
)
    
SELECT t1.education, (t1.ttl_diabetic / t3.ttl_smoker) as perc
from t1, t3

,education,perc
0,1,0.000849
1,2,0.021368
2,3,0.041471
3,4,0.199877
4,5,0.187017
5,6,0.187848


### 5. Some hospitals had approached you and are asking for some information. Please help them get the following -
    1. Please generate a table of each education level, and the top 5 persons (by ID), where the persons are ranked according to their BMI, DESC
    2. Let’s create a new metric per each person, called “BMIToMentalActivityRatio” - calculated as: BMI / mental activity.

In [22]:
-- Please generate a table of each education level, and the top 5 persons (by ID), where the persons are ranked according to their BMI, DESC
WITH t1 AS (
SELECT d.uniqueid as uniqueid, d.education as education, bmi, 
ROW_NUMBER() OVER (PARTITION BY education ORDER BY bmi DESC) as rank
FROM demographic_data d
JOIN id_label i
ON d.uniqueid = i.uniqueid
JOIN health_measures h
ON i.uniqueid = h.uniqueid
JOIN enriched_data e
ON h.uniqueid = e.uniqueid
)
SELECT uniqueid, education, bmi
FROM t1
WHERE rank = 1
ORDER BY 3 DESC

,uniqueid,education,bmi
0,79478,4,98
1,76396,5,98
2,76394,6,98
3,79522,3,95
4,79543,2,84
5,100832,1,57


In [24]:
-- Let’s create a new metric per each person, called “BMIToMentalActivityRatio” - calculated as: BMI / mental activity.

WITH t1 AS (
SELECT d.uniqueid as uniqueid, income, (h.bmi/h.menthlth) AS BMIToMentalActivityRatio
FROM demographic_data d
JOIN id_label i
ON d.uniqueid = i.uniqueid
JOIN health_measures h
ON i.uniqueid = h.uniqueid
JOIN enriched_data e
ON h.uniqueid = e.uniqueid
WHERE h.menthlth != 0
ORDER BY 3 DESC
)
SELECT DISTINCT income, uniqueid, BMIToMentalActivityRatio
FROM (
    SELECT income, uniqueid, BMIToMentalActivityRatio, 
    row_number() OVER (PARTITION BY income ORDER BY BMIToMentalActivityRatio DESC) AS ranks
    FROM t1
    GROUP BY 1, 2, 3
) ranked
WHERE ranks = 1
GROUP BY 1, 2, 3
ORDER BY 1, 3 DESC;

,income,uniqueid,bmitomentalactivityratio
0,1,213505,44
1,2,58641,56
2,3,79327,79
3,4,79453,73
4,5,76369,79
5,6,79386,92
6,7,79340,89
7,8,79516,87
